In [1]:
# import package
from __future__ import print_function
import pandas as pd
from pandas import DataFrame
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import time

Using TensorFlow backend.


In [2]:
# format data and define parameters
train = pd.read_csv("mnist_train.csv")
test = pd.read_csv("mnist_test.csv")
train = train.sample(frac=1).reset_index(drop=True)
# define
img_rows, img_cols = 28, 28
n_classes = 10
batch_size = 128
trainning_period = 20
learning_rate = 1e-3
# format the data
train_x = train.iloc[:,1:].as_matrix().reshape(train.shape[0], img_rows, img_cols)
train_y_array = train.iloc[:,0]
train_y = keras.utils.to_categorical(train_y_array, n_classes)

test_x = test.iloc[:,1:].as_matrix().reshape(test.shape[0], img_rows, img_cols)
test_y_array = test.iloc[:,0]
test_y = keras.utils.to_categorical(test_y_array, n_classes)

if K.image_data_format() == 'channels_first':
    train_x = train_x.reshape(train_x.shape[0], 1, img_rows, img_cols)
    test_x = test_x.reshape(test_x.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    train_x = train_x.reshape(train_x.shape[0], img_rows, img_cols, 1)
    test_x = test_x.reshape(test_x.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [4]:
print(train_x.shape)

(10000, 28, 28, 1)


In [5]:
# build network
model = Sequential()
# add first convolutional layer
model.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',padding='valid',
                 input_shape=input_shape))
# add maxpooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
# add second convolutional layer
model.add(Conv2D(64, kernel_size=(5, 5),
                 activation='relu',padding='valid',
                 input_shape=input_shape))
# add maxpooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
# add a fully connected layer
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
# add a dropout rate
model.add(Dropout(0.2))
# add final layer 
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.Adam(lr=learning_rate,decay=0.01),
              metrics=['accuracy'])

In [6]:
# fit the model
time_0 = time.time()
model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=trainning_period,
          verbose=1,
          validation_data=(test_x, test_y))
print("total time is:", time.time() - time_0)

Train on 10000 samples, validate on 2000 samples
Epoch 1/20
10000/10000 [==============================] - 2s - loss: 0.5419 - acc: 0.8360 - val_loss: 0.1672 - val_acc: 0.9465
Epoch 2/20
10000/10000 [==============================] - 0s - loss: 0.1467 - acc: 0.9578 - val_loss: 0.1020 - val_acc: 0.9645
Epoch 3/20
10000/10000 [==============================] - 0s - loss: 0.1017 - acc: 0.9699 - val_loss: 0.0696 - val_acc: 0.9795
Epoch 4/20
10000/10000 [==============================] - 0s - loss: 0.0761 - acc: 0.9772 - val_loss: 0.0557 - val_acc: 0.9835
Epoch 5/20
10000/10000 [==============================] - 0s - loss: 0.0617 - acc: 0.9824 - val_loss: 0.0437 - val_acc: 0.9875
Epoch 6/20
10000/10000 [==============================] - 0s - loss: 0.0513 - acc: 0.9857 - val_loss: 0.0385 - val_acc: 0.9900
Epoch 7/20
10000/10000 [==============================] - 0s - loss: 0.0475 - acc: 0.9866 - val_loss: 0.0332 - val_acc: 0.9900
Epoch 8/20
10000/10000 [==============================] - 0s -

In [7]:
# predict probablity of test data
pred_prob = model.predict_proba(test_x)
print(pred_prob)
# predict class of test data
pred_class = model.predict_classes(test_x)
print(pred_class)

1344/2000 [===================>..........] - ETA: 0s[[  9.99717057e-01   1.55675339e-09   2.81863875e-04 ...,   3.51404961e-10
    1.07107789e-08   1.03072159e-06]
 [  9.99980092e-01   9.64977473e-11   3.54426993e-06 ...,   1.55820301e-09
    7.24995664e-09   1.26439509e-05]
 [  9.99162555e-01   7.00401017e-08   2.60509289e-04 ...,   3.99985845e-04
    1.28167937e-06   1.49774482e-04]
 ..., 
 [  3.85989990e-10   7.28584727e-13   8.50315790e-11 ...,   4.08214237e-06
    2.05817059e-07   9.99993682e-01]
 [  4.52355486e-10   1.32570702e-14   3.08022829e-10 ...,   2.31452006e-07
    3.85992536e-08   9.99999285e-01]
 [  2.98005023e-11   2.41544011e-08   8.01681499e-11 ...,   4.34523872e-06
    1.31381341e-04   9.99797881e-01]]
1248/2000 [=================>............] - ETA: 0s[0 0 0 ..., 9 9 9]
